In [1]:
from pyspark import SparkContext
from pyspark import SparkConf


In [2]:
from itertools import chain, combinations
import copy

In [3]:
conf = SparkConf().setAppName("apriori")
sc = SparkContext(conf=conf)

In [18]:
S = 100
freqSetDict = {}

In [5]:
text = sc.textFile('browsing.txt')

In [6]:
text.take(5)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ',
 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ',
 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ']

# k = 1

In [7]:
candSetCount1 = text.flatMap(lambda t: t.strip().split(' ')) \
    .map(lambda item: (item, 1)).reduceByKey(lambda x,y: x+y )

In [14]:
# get frequent set
freqSetCount1 = candSetCount1.filter(lambda x: x[1] >= S)

In [15]:
freqSet1 = freqSetCount1.map(lambda x: x[0])

In [19]:
freqSetDict.update(freqSetCount1.collect())

In [109]:
# generate association rules
# compute confidence score

def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

def getAssociationRule(item, suppDict,k):
    """
    item: tuple, k-item frequent 
    suppDict: itemSuppDict
    return rules: Association Rule
    """
    _item = set(item)
    rules = []
    _subsets = map(frozenset, [x for x in subsets(_item) if len(x) < k])
    for i in _subsets:
        rightItem = i
        leftItem = tuple(_item.difference(rightItem))
        leftItem = leftItem if len(leftItem) > 1 else leftItem[0]
        conf = suppDict[item]/suppDict[leftItem]
        r = ''.join(leftItem) + '|' + ','.join(rightItem)
        rules.append((r, conf))
        print("item", item)
        print("leftItem",leftItem)
    return rules
            

In [23]:
textlist = text.map(lambda x: x.strip().split(' '))

# k = 2


In [25]:
# k = 2
candSet2 = freqSet1.cartesian(freqSet1).filter(lambda x: x[0] < x[1] ) 

In [26]:
# count support
candSetCount22 = candSet2.cartesian(textlist) \
#             .map(lambda item: (item[0],1) if set(item[0]).issubset(item[1]) else None) \
#             .filter(lambda x: x != None) \
#             .reduceByKey(lambda x,y: x+y) 

In [27]:
candSetCount2 = candSetCount22.map(lambda item: (item[0],1) if set(item[0]).issubset(item[1]) else None) \
            .filter(lambda x: x != None) \
            .reduceByKey(lambda x,y: x+y) 

In [28]:
freqSetCount2 = candSetCount2.filter(lambda x: x[1] >= S)

In [29]:
# update freqSetDict
freqSetDict.update(freqSetCount2.collect())

In [30]:
freqSet2 = freqSetCount2.map(lambda x: x[0])

In [31]:
rules2 = freqSet2.flatMap(lambda x: getAssociationRule(x, freqSetDict, 2))

In [32]:
# top 5 pairs
newRules2 = rules2.sortBy(lambda t: (-t[1],t[0]))

In [112]:
newRules2.take(5)

[('DAI93865|FRO40251', 1.0),
 ('GRO85051|FRO40251', 0.999176276771005),
 ('GRO38636|FRO40251', 0.9906542056074766),
 ('ELE12951|FRO40251', 0.9905660377358491),
 ('DAI88079|FRO40251', 0.9867256637168141)]

# k = 3

In [40]:
candSet = freqSet2.cartesian(freqSet2).filter(lambda x: x[0] < x[1] ) 

In [41]:
candSet.take(5)

[(('GRO73461', 'SNA72163'), ('GRO73461', 'SNA80324')),
 (('GRO73461', 'SNA72163'), ('SNA24799', 'SNA55617')),
 (('GRO73461', 'SNA72163'), ('SNA44190', 'SNA80324')),
 (('GRO73461', 'SNA72163'), ('SNA45677', 'SNA93860')),
 (('GRO73461', 'SNA72163'), ('GRO81087', 'SNA45677'))]

In [59]:
def getCandSet(item, k, freqDict):
    t1 = set(item[0])
    t2 = set(item[1])
    t = t1.union(t2)
    if len(t) == k and tuple(t.difference(t1).union(t.difference(t2))) in freqDict:
        return t
    return None

In [61]:
_candSet = candSet.map(lambda x: getCandSet(x,3,freqSetDict)).filter(lambda x: x)
_candSet.take(5)

[{'GRO73461', 'SNA44190', 'SNA80324'},
 {'DAI55911', 'GRO73461', 'SNA80324'},
 {'ELE12792', 'GRO73461', 'SNA80324'},
 {'ELE11111', 'ELE32164', 'SNA80324'},
 {'DAI55911', 'ELE32164', 'SNA80324'}]

In [62]:
len(_candSet.collect())

9246

In [74]:
def countSupp(item):
    t1= item[0]
    t2 = item[1]
    print("t1",t1)
    print("t2",t2)
    if t1.issubset(t2):
        print("true",(item[0],1))
    return None
# for x in candSetCount22.take(5):
#     countSupp(x)
x = ({'SNA80324', 'SNA44190', 'GRO73461'}, ['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222','SNA80324', 'SNA44190', 'GRO73461'])
countSupp(x)

t1 {'SNA80324', 'SNA44190', 'GRO73461'}
t2 ['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222', 'SNA80324', 'SNA44190', 'GRO73461']
true ({'SNA80324', 'SNA44190', 'GRO73461'}, 1)


In [78]:
_candSet = candSet.map(lambda x: getCandSet(x,3,freqSetDict)).filter(lambda x: x)
_candSet.take(5)
candSetCount  = candSetCount22.map(lambda item: (tuple(item[0]),1) if item[0].issubset(item[1]) else None) \
        .filter(lambda x: x) \
        .reduceByKey(lambda x,y: x+y) 
candSetCount.take(5)

[(('SNA80324', 'ELE12792', 'ELE17451'), 55),
 (('ELE66810', 'ELE32164', 'FRO80039'), 34),
 (('SNA80324', 'ELE74009', 'GRO30386'), 16),
 (('SNA80324', 'ELE56788', 'GRO30386'), 15),
 (('GRO71621', 'DAI85309', 'SNA40784'), 46)]

In [79]:
freqSetCount = candSetCount.filter(lambda x: x[1] >= S)
freqSetCount.take(5)

[(('DAI75645', 'SNA80324', 'SNA90094'), 231),
 (('DAI62779', 'GRO15017', 'ELE17451'), 222),
 (('DAI62779', 'GRO15017', 'ELE26917'), 132),
 (('FRO78087', 'DAI62779', 'GRO59710'), 114),
 (('GRO56726', 'DAI62779', 'GRO73461'), 178)]

In [81]:
# update freqSetDict
suppDict.update(freqSetCount.collect())

In [83]:
freqSet = freqSetCount.map(lambda x: x[0])
freqSet.take(5)

[('DAI75645', 'SNA80324', 'SNA90094'),
 ('DAI62779', 'GRO15017', 'ELE17451'),
 ('DAI62779', 'GRO15017', 'ELE26917'),
 ('FRO78087', 'DAI62779', 'GRO59710'),
 ('GRO56726', 'DAI62779', 'GRO73461')]

In [101]:
rules = freqSet.flatMap(lambda x: getAssociationRule(x, suppDict, k))
rules.take(10)

[('SNA80324SNA90094|DAI75645', 1.5),
 ('DAI75645SNA90094|SNA80324', 1.0744186046511628),
 ('DAI75645SNA80324|SNA90094', 0.20442477876106194),
 ('SNA90094|DAI75645,SNA80324', 0.16618705035971224),
 ('SNA80324|DAI75645,SNA90094', 0.07588699080157688),
 ('DAI75645|SNA80324,SNA90094', 0.08442982456140351),
 ('DAI62779ELE17451|GRO15017', 0.13944723618090452),
 ('DAI62779GRO15017|ELE17451', 0.5677749360613811),
 ('ELE17451|DAI62779,GRO15017', 0.05729032258064516),
 ('DAI62779|GRO15017,ELE17451', 0.03329833508324584),
 ('GRO15017|DAI62779,ELE17451', 0.17411764705882352),
 ('DAI62779ELE26917|GRO15017', 0.20307692307692307),
 ('DAI62779GRO15017|ELE26917', 0.3375959079283887),
 ('ELE26917|DAI62779,GRO15017', 0.05759162303664921),
 ('DAI62779|GRO15017,ELE26917', 0.019799010049497526),
 ('GRO15017|DAI62779,ELE26917', 0.10352941176470588),
 ('DAI62779GRO59710|FRO78087', 0.20320855614973263),
 ('FRO78087GRO59710|DAI62779', 0.8507462686567164),
 ('GRO59710|FRO78087,DAI62779', 0.05688622754491018),
 (

In [94]:
freqSetDict[('GRO15017', 'ELE17451')]

KeyError: ('GRO15017', 'ELE17451')

In [103]:
# top 5 pairs
newRules = rules.sortBy(lambda t: (-t[1],t[0]))
newRules.take(5)

[('GRO85051SNA80324|FRO40251', 3.0),
 ('DAI88079GRO73461|FRO40251', 2.9793103448275864),
 ('DAI88079SNA80324|FRO40251', 2.936619718309859),
 ('DAI23334ELE17451|DAI62779', 2.91),
 ('ELE99737FRO19221|DAI85309', 2.5037593984962405)]